## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-16-53-51 +0000,nypost,"DOJ teases details about Jeffrey Epstein, Ghis...",https://nypost.com/2025/07/30/us-news/doj-urge...
1,2025-07-30-16-49-02 +0000,startribune,Minnesota eases ‘forever chemicals’ ban to exe...,https://www.startribune.com/minnesota-eases-pf...
2,2025-07-30-16-48-59 +0000,wapo,Pilot of Black Hawk helicopter had received hi...,https://www.washingtonpost.com
3,2025-07-30-16-48-59 +0000,wapo,NTSB hearing live updates: Black Hawk devices ...,https://www.washingtonpost.com/business/2025/0...
4,2025-07-30-16-48-57 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/07/30/us/dc-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,59
238,tsunami,22
395,new,20
97,gaza,18
193,palestinian,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
252,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,123
197,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,119
71,2025-07-30-14-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,117
261,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...,110
333,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
252,123,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
31,69,2025-07-30-16-08-35 +0000,wapo,"Tsunami live updates: Waves reach Hawaii, Cali...",https://www.washingtonpost.com/world/2025/07/2...
275,67,2025-07-29-21-18-23 +0000,wapo,New details emerge in New York City shooting i...,https://www.washingtonpost.com/nation/2025/07/...
197,60,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
315,52,2025-07-29-18-32-06 +0000,nypost,UK will recognize Palestinian state unless Isr...,https://nypost.com/2025/07/29/world-news/uk-wi...
155,52,2025-07-30-09-30-00 +0000,wsj,The shake-up for charitable giving inside the ...,https://www.wsj.com/politics/policy/trump-tax-...
32,49,2025-07-30-16-06-00 +0000,wsj,Any further Bank of Canada rate reduction depe...,https://www.wsj.com/articles/bank-of-canada-st...
19,47,2025-07-30-16-23-00 +0000,wsj,Two top Senate Republicans laid out a plan Wed...,https://www.wsj.com/politics/policy/us-weapons...
107,44,2025-07-30-12-43-30 +0000,nypost,Here’s what will happen to Shane Tamura’s brai...,https://nypost.com/2025/07/30/us-news/nyc-gunm...
268,43,2025-07-29-21-30-24 +0000,nypost,Ukrainian mom-to-be expecting first child afte...,https://nypost.com/2025/07/29/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
